In [24]:
import tensorflow as tf
print("TensorFlow devices:", tf.config.list_physical_devices())

tf.debugging.set_log_device_placement(True)


TensorFlow devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [12]:
# %pip install kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gpiosenka/cards-image-datasetclassification")

print("Path to dataset files:", path)

Path to dataset files: /Users/ajitesh/.cache/kagglehub/datasets/gpiosenka/cards-image-datasetclassification/versions/2


In [13]:
# %pip install tensorflow numpy pandas matplotlib seaborn scikit-learn
from tensorflow import keras 
from keras.models import Model
from keras.layers import Dense, Dropout, BatchNormalization, Input
import os
import pandas as pd

In [14]:

from keras.utils import load_img
import glob

# Get all jpg image paths in the dataset directory (including subfolders)
image_paths = glob.glob(os.path.join(path, '**', '*.jpg'), recursive=True)

# Load images into a list
# train_data = [load_img(img_path) for img_path in image_paths]
train_data = []
labels = []
for i in image_paths:
    train_data.append(load_img(i))
    labels.append(i.split('/')[-2])  # Assuming the label is the folder name

train_data[1].size
# Convert to DataFrame
train_df = pd.DataFrame({
    'image': train_data,
    'label': labels
})
train_df.head()
# convert jpg images to tensors
from keras.preprocessing.image import img_to_array
train_df['image'] = train_df['image'].apply(lambda x: img_to_array(x))



In [15]:
import numpy as np
# Convert labels to categorical format
coded_labels={
"ace of clubs" :1       ,    "eight of hearts" :2  ,     "four of clubs" :3  ,          "jack of hearts" :4   ,       "king of spades" :5  ,        "queen of diamonds" :6       , "seven of spades" :7   , "ten of diamonds" :8         , "three of spades" :9,
"ace of diamonds" :10     ,   "eight of spades" :11     ,    "four of diamonds" :12 ,       "jack of spades" :13   ,       "nine of clubs" :14    ,       "queen of hearts" :15         , "six of clubs" :16            , "ten of hearts" :17           , "two of clubs" :18,
"ace of hearts" :19       ,   "five of clubs" :20       ,    "four of hearts" :21 ,         "joker" :22           ,        "nine of diamonds" :23 ,       "queen of spades" :24         , "six of diamonds" :25         , "ten of spades" :26           , "two of diamonds" :27,
"ace of spades" :28       ,   "five of diamonds" :29    ,    "four of spades" :30 ,         "king of clubs" :31   ,        "nine of hearts" :32   ,       "seven of clubs" :33          , "six of hearts" :34           , "three of clubs" :35          , "two of hearts" :36,
"eight of clubs" :37      ,   "five of hearts" :38      ,    "jack of clubs" :39 ,          "king of diamonds" :40   ,     "nine of spades" :41   ,      "seven of diamonds" :42       , "six of spades" :43           , "three of diamonds" :44       , "two of spades" :45,
"eight of diamonds" :46   ,   "five of spades" :47      ,    "jack of diamonds" :48 ,       "king of hearts" :49   ,       "queen of clubs" :50   ,      "seven of hearts" :51        , "ten of clubs" :52            , "three of hearts" :53
}
def label_to_one_hot(label):
    one_hot = np.zeros(53)
    one_hot[coded_labels[label]-1] = 1
    return one_hot

y_train = np.array([label_to_one_hot(label) for label in train_df['label']])
y_train


array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
x_train = np.array(train_df['image'].tolist())
x_train

array([[[[236., 221., 214.],
         [235., 220., 213.],
         [219., 204., 199.],
         ...,
         [220., 203., 195.],
         [220., 203., 193.],
         [221., 205., 192.]],

        [[242., 227., 220.],
         [240., 225., 218.],
         [218., 203., 196.],
         ...,
         [219., 202., 194.],
         [220., 203., 193.],
         [221., 205., 192.]],

        [[236., 221., 214.],
         [239., 224., 217.],
         [218., 203., 196.],
         ...,
         [220., 201., 194.],
         [221., 203., 193.],
         [222., 204., 194.]],

        ...,

        [[186., 175., 173.],
         [190., 179., 173.],
         [199., 186., 177.],
         ...,
         [228., 209., 177.],
         [226., 210., 177.],
         [225., 209., 176.]],

        [[187., 176., 180.],
         [185., 175., 176.],
         [187., 176., 170.],
         ...,
         [228., 208., 175.],
         [231., 207., 173.],
         [231., 205., 170.]],

        [[190., 180., 188.],
       

In [17]:
x_train[1].shape
# take only (1/4,1/4) portion of the image since card can be classified by the top left corner
x_train = x_train[:, :(int(x_train.shape[1]/4)), :(int(x_train.shape[2]/4)), :]

In [18]:
#override the keras.models . Dense ....to add output from the dense layer is feeded to a variable...ie every Dense layer output is stored in a variable
outputs_of_all_dense_layers = []
def Dense(*args, **kwargs):
    layer = keras.layers.Dense(*args, **kwargs)
    def wrapper(input_tensor):
        output_tensor = layer(input_tensor)
        outputs_of_all_dense_layers.append(output_tensor)
        return output_tensor
    return wrapper


In [19]:
# input_layer = Input(shape=(224, 224, 3))
input_layer = Input(shape=(int(x_train.shape[1]), int(x_train.shape[2]), 3))
covn_layer = keras.layers.Conv2D(32, (3, 3), activation='relu')(input_layer)
# covn_layer2 = keras.layers.MaxPooling2D((2, 2))(covn_layer)
flatten_layer = keras.layers.Flatten()(covn_layer)
hidden_layer_1 = Dense(128, activation='relu')(flatten_layer)
hidden_layer_3 = Dense(53, activation='softmax')(hidden_layer_1)
model=Model(inputs=input_layer, outputs=hidden_layer_3)
model.summary()

I0000 00:00:1755961838.393616  293811 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1755961838.393644  293811 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 56, 56, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 54, 54, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 93312)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,944,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 53)             │         6,837 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,951,797 (45.59 MB)

 Trainable params: 11,951,797 (45.59 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
import time 
start_time = time.time()
history1 = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
end_time = time.time()
print(f"Training time: {end_time - start_time} seconds")

Epoch 1/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.9560 - loss: 73.2773 - val_accuracy: 0.0196 - val_loss: 20170.7031
Epoch 2/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9604 - loss: 69.4830 - val_accuracy: 0.0190 - val_loss: 20485.4863
Epoch 3/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9609 - loss: 63.0722 - val_accuracy: 0.0178 - val_loss: 19496.6465
Epoch 4/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9623 - loss: 84.5709 - val_accuracy: 0.0239 - val_loss: 23561.5059
Epoch 5/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9594 - loss: 81.0832 - val_accuracy: 0.0307 - val_loss: 19852.2480
Epoch 6/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9525 - loss: 118.6111 - val_accuracy: 0.0123 - val_loss: 23890.3984
Epoch 7/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9598 - loss: 84.9818 - val_accuracy: 0.0190 - val_loss: 22411.5488
Epoch 8/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accura

In [ ]:
model.get_weights()

In [ ]:
outputs_of_all_dense_layers
# print (outputs_of_all_dense_layers) for all layers
print (outputs_of_all_dense_layers[-1])  # for last dense layer
# <KerasTensor shape=(None, 53), dtype=float32, sparse=False, ragged=False, name=keras_tensor_34> print this